This notebook receives as input a list of simulation_ids and determines all the champions.

The goal is to determine the feasibility of our simulations compared to the real champions. We expect that the actual champions are the most frequent outcome.

In [3]:
import binarytree as bt
import sqlalchemy
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [77]:
engine = sqlalchemy.create_engine('sqlite:///../nba_manager.db')

In [78]:
SCENARIO_GROUP_ID = '716bb89f-baa4-43b2-bf9b-407f5be6fe4a'

In [82]:
scenarios = pd.read_sql_query('select * from scenarios where scenario_group_id="{}"'.format(SCENARIO_GROUP_ID), engine)
playoff_brackets = pd.read_sql_table('playoff_brackets', engine)
playoff_brackets.sort_values('id', ascending=False, inplace=True)
teams = pd.read_sql_table('teams', engine)
#simulation_id = playoff_brackets.iloc[0]['simulation_id']
#simulation_year = playoff_brackets.iloc[0]['year']
#nodes_sep_comma = playoff_brackets.iloc[0]['nodes_sep_comma']

In [83]:
teams[teams.id==14].head()

,id,name,short_name,conference,division
13,14,Washington Wizards,WAS,EAST,SOUTHEAST


In [84]:
simulation_ids = scenarios.simulation_id.unique().tolist()
print (len(simulation_ids))

10


In [85]:
def get_champions(simulation_ids):
    tuples = []
    playoff_brackets = pd.read_sql_table('playoff_brackets', engine)
    playoff_brackets = playoff_brackets[playoff_brackets.simulation_id.isin(simulation_ids)]
    for element in playoff_brackets.itertuples():
        champion_id = element.nodes_sep_comma.split(',')[0]
        tuples.append((element.year, int(champion_id)))
    
    return tuples

In [86]:
#get_champions(simulation_ids)

In [87]:
champion_ids = [i[1] for i in get_champions(simulation_ids)]
champion_df = pd.DataFrame(dict(champion_id=champion_ids))
champion_df_plus = pd.merge(champion_df, teams, left_on='champion_id', right_on='id', copy=True)
champion_df_plus.name.value_counts()

Washington Wizards        10
Toronto Raptors            5
Los Angeles Lakers         5
Denver Nuggets             4
Memphis Grizzlies          4
Indiana Pacers             4
Charlotte Hornets          3
Minnesota Timberwolves     3
Boston Celtics             3
Los Angeles Clippers       2
Oklahoma City Thunder      1
Detroit Pistons            1
Phoenix Suns               1
Orlando Magic              1
San Antonio Spurs          1
Sacramento Kings           1
Milwaukee Bucks            1
Name: name, dtype: int64